<a href="https://colab.research.google.com/github/valerie-lin/MangoStyle/blob/master/ResNeXT_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

把 drive 的檔案叫過來，這邊不用跑這個block

In [0]:
from google.colab import drive
import os
drive.mount('/content/gdrive')

### 用 ID 得到C1-P1_Train 再解壓縮
* 路徑在 /content
* 等等執行時會直接在這個目錄下

In [0]:
!gdown --id '1euWEtsOIrF0H9Ob9wjtn5Oo5KmlmEeHM' --output C1-P1_Train\ Dev.rar
!unrar x C1-P1_Train\ Dev.rar
!ls

In [0]:
#https://drive.google.com/file/d/1WTI8HzfuqWyH3IIeLG955LUgmnjISzgS/view?usp=sharing
#https://drive.google.com/file/d/1pL7uF5Ej-IWYwdcXG37JSQFTUnFWgE07/view?usp=sharing

!gdown --id '1WTI8HzfuqWyH3IIeLG955LUgmnjISzgS' --output Dev.tar.xz
!tar -xvf  'Dev.tar.xz' 

!gdown --id '1pL7uF5Ej-IWYwdcXG37JSQFTUnFWgE07' --output train.tar.xz
!tar -xvf  'train.tar.xz'
!ls



In [0]:
!pip install -r requirements.txt



import


In [0]:
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo
import torchvision.models as models
import torch
import cv2
from PIL import Image
import torchvision.transforms as transforms
from torch.autograd import Variable

import pandas as pd
import numpy as np
from pathlib import Path

In [0]:
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
import pickle
import sklearn

用 torch.hub 上的 ResNeXT 50

In [0]:
print('The torch version is {}.'.format(torch.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))
def makeModel():
	model = torch.hub.load('pytorch/vision:v0.5.0', 'resnext50_32x4d', pretrained=True)
	model.eval()
	return model

The torch version is 1.4.0.
The scikit-learn version is 0.21.2.


In [0]:
def extractFeature(model, dataSetName, imgPath, numData):
	labelName = Path(dataSetName).stem + 'Label.npy'
	featureName = Path(dataSetName).stem + 'Feature.npy'
	print(labelName)
	print(featureName)
	transform_list = [
		transforms.Resize((224, 224)),
		transforms.RandomRotation(degrees=15),
		transforms.RandomHorizontalFlip(p=0.5),
		transforms.ToTensor(),
		transforms.Normalize(mean=[0.485, 0.456, 0.406],std =[0.229, 0.224, 0.225])
	]
	img_to_tensor = transforms.Compose(transform_list)
	#df = pd.read_csv('C1-P1_Train-Dev/train.csv')
	#df = pd.read_csv('C1-P1_Train-Dev/dev.csv')
	df = pd.read_csv(dataSetName, header=0)
	data_npy = df.to_numpy()
	print(data_npy)

	#path = 'C1-P1_Train-Dev/C1-P1_Train/'
	path = imgPath

	feature = []
	label = []
	Total = len(data_npy)
	count = 1
	if(numData < Total):
		Total = numData

	for data in data_npy[0:Total]:
		print('{}/{}'.format(count, Total))
		count += 1
		print(data)
		imgName = path + data[0]
		print(imgName)
		print(data[1])
		img = Image.open(imgName)

		tensor=img_to_tensor(img)
		tensor=tensor.resize_(1,3,224,224)

		result = model(Variable(tensor))
		result_npy = result.data.numpy()
		#print(result.size())
		#print(result_npy.shape)

		feature.append(result_npy)
		label.append(data[1])

		#cv2.imshow("", img)
		#cv2.waitKey(0)
		#cv2.destroyAllWindows()
	label = np.array(label)
	feature = np.array(feature)

	np.save(labelName, label)
	np.save(featureName, feature)


In [0]:
if __name__ == '__main__':
	model = makeModel()
	device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
	dataSet = input('1:train 2:val ')
	numData = int(input('desired number of data: '))
	if(dataSet == '1'):
		dataSetName = 'train_1.csv'
		imgPath = 'train_selected/'
	elif(dataSet == '2'):
		dataSetName = 'dev_1.csv'
		imgPath = 'dev_selected/'
	else:
		print('wrong choose')
		exit(0)
	extractFeature(model, dataSetName, imgPath, numData)


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.5.0


1:train 2:val 2
desired number of data: 800
dev_1Label.npy
dev_1Feature.npy
[['00027.jpg' 'B']
 ['00033.jpg' 'A']
 ['00047.jpg' 'B']
 ...
 ['07970.jpg' 'C']
 ['07989.jpg' 'A']
 ['07995.jpg' 'C']]
1/800
['00027.jpg' 'B']
dev_selected/00027.jpg
B
2/800
['00033.jpg' 'A']
dev_selected/00033.jpg
A
3/800
['00047.jpg' 'B']
dev_selected/00047.jpg
B
4/800
['00050.jpg' 'B']
dev_selected/00050.jpg
B
5/800
['00051.jpg' 'C']
dev_selected/00051.jpg
C
6/800
['00052.jpg' 'B']
dev_selected/00052.jpg
B
7/800
['00059.jpg' 'C']
dev_selected/00059.jpg
C
8/800
['00060.jpg' 'A']
dev_selected/00060.jpg
A
9/800
['00069.jpg' 'B']
dev_selected/00069.jpg
B
10/800
['00070.jpg' 'A']
dev_selected/00070.jpg
A
11/800
['00073.jpg' 'A']
dev_selected/00073.jpg
A
12/800
['00078.jpg' 'B']
dev_selected/00078.jpg
B
13/800
['00082.jpg' 'C']
dev_selected/00082.jpg
C
14/800
['00111.jpg' 'B']
dev_selected/00111.jpg
B
15/800
['00119.jpg' 'C']
dev_selected/00119.jpg
C
16/800
['00129.jpg' 'B']
dev_selected/00129.jpg
B
17/800
['0013

TrainSVM

In [0]:
def loadDataSet(Xpath, Ypath):
    X = np.load(Xpath)
    X = np.array([a[0] for a in X])
    Y = np.load(Ypath)
    return X, Y

def constructSVC(C,P):
    clf = Pipeline([('anova', SelectPercentile(f_classif)),
                    ('svc', SVC(C=1.0, kernel="linear", class_weight='balanced', verbose=True))])
    percentile = P
    clf.set_params(anova__percentile=percentile)
    return clf

def train(clf, X_train,y_train):
    clf.fit(X_train,y_train)
    return clf



In [0]:
if __name__ == '__main__':
    Train_Xpath = 'train_1Feature.npy'
    Train_Ypath = 'train_1Label.npy'
    Test_Xpath = 'dev_1Feature.npy'
    Test_Ypath = 'dev_1Label.npy'
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    X_train, y_train = loadDataSet(Train_Xpath, Train_Ypath)
    print(X_train.shape)
    print(y_train.shape)
    #X_test, y_test = loadDataSet(Test_Xpath, Test_Ypath)
    #X_train, X_test, y_train, y_test = splitTrainTest(X,Y)

    percentile = 20
    C = 1.0
    clf = constructSVC(C,percentile)
    print('start Training')
    clf = train(clf, X_train, y_train)

    with open('clf.pickle', 'wb') as f:
        pickle.dump(clf, f)

(5600, 1000)
(5600,)
start Training
[LibSVM]

PredictSVM

In [0]:
import numpy as np
import pickle
from sklearn.metrics import recall_score

def loadDataSet(Xpath, Ypath):
    X = np.load(Xpath)
    X = np.array([a[0] for a in X])
    Y = np.load(Ypath)
    return X, Y

def predict(clf, X_test):
    print(clf)
    #this_scores = clf.score(X_test, y_test)
    y_pred = clf.predict(X_test)
    return y_pred

def evaluate(y_test, y_pred):
    this_scores = recall_score(y_test, y_pred, average='weighted')
    return this_scores

def evaluate_scratch(y_test, y_pred):
    A = list(zip(y_pred,y_test))
    #print(A)
    #print('all num:', len(A))
    diff = len([a for a in A if (a[0] != a[1])])
    same = len([a for a in A if (a[0] == a[1])])
    print('diff num:', diff, 'same num:', same)
    return same/len(A)

In [0]:
if __name__ == '__main__':
    Test_Xpath = 'dev_1Feature.npy'
    Test_Ypath = 'dev_1Label.npy'
    X_test, y_test = loadDataSet(Test_Xpath, Test_Ypath)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    with open('clf.pickle', 'rb') as f:
        clf = pickle.load(f)
        y_pred = predict(clf, X_test)
        recall = evaluate(y_test, y_pred)
        #score = clf.score(X_test, y_test)
        print('accuracy:', recall)

Pipeline(memory=None,
         steps=[('anova',
                 SelectPercentile(percentile=20,
                                  score_func=<function f_classif at 0x7f56f9079ea0>)),
                ('svc',
                 SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
                     decision_function_shape='ovr', degree=3,
                     gamma='auto_deprecated', kernel='linear', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=True))],
         verbose=False)
accuracy: 0.74
